# <center> <b> <h1> Random Forest (Permutation-test) </h1></b></center> 


In [2]:
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
import joblib
from sklearn.metrics import confusion_matrix
import pandas as pd

from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.metrics import classification_report
import itertools
import random

from concurrent import futures
from concurrent.futures import ProcessPoolExecutor

import numpy as np 

PATH_DATASETS = "../../2_Data_preparation/2_Split_Raw_Datasets/output/"
PATH_PKL = "../../3.5 Modeling/1_2_hyper_parameter_tunning/output/"
PATH_SIGNIFICANCE = "../../3 Modeling/2. Random Forest (significance Score)/output/"

In [3]:
def readCsvs(split):
    datasets = []
    constant1 = 'X_'
    constant2 = '_' + split
    features = ['area', 'meancurv', 'thickness', 'volume']

    for feature in features:
        df = pd.read_csv(PATH_DATASETS + constant1 + feature + constant2 +'.csv', index_col=0)
        datasets.append({'name': feature, 'data': df})
    return datasets

In [4]:
def number_of_rois(name):
    
    df_number_of_features = pd.read_csv(PATH_SIGNIFICANCE 
                                        + "Random Forest Best Number Of Features.csv", index_col=0)
    
    features = ['area','meancurv', 'thickness', 'volume']
    number_of_feature = df_number_of_features['quantity'].values

    x = 0 
    result = 0 
    for feature in features:
        if feature == name:
             result = int(number_of_feature[x])
        x += 1
    return result

In [5]:
def readSignificances():
    df_number_of_features = pd.read_csv(PATH_SIGNIFICANCE 
                                        + "Random Forest Best Number Of Features.csv", index_col=0)
    
    datasets = []
    constant1 = 'Random Forest (AllFeatures - Significance - '
    constant2 = ')'
    features = ['area','meancurv', 'thickness', 'volume']
    number_of_feature = df_number_of_features['quantity'].values

    for feature, number in zip(features, number_of_feature):
        df = pd.read_csv(PATH_SIGNIFICANCE + constant1 + str(int(number)) + constant2 
                         + feature +'.csv', index_col=0)
        datasets.append({'name': feature[0], 'data': df})
    return datasets

# 1. Prepare the data

In [6]:
Significances = readSignificances()
Significances[0]['data'].head()

,Significance
Name,
rh_G_and_S_cingul-Ant_area,0.020898
lh_S_temporal_sup_area,0.018332
rh_G_temporal_middle_area,0.017275
rh_S_temporal_sup_area,0.015924
lh_G_cuneus_area,0.015085


In [7]:
df_X_val = readCsvs('validate')
df_X_val[0]['data'].shape

(338, 148)

In [8]:
df_y_val = pd.read_csv(PATH_DATASETS + 'Y_validate.csv', index_col = 0)
df_y_vals = [df_y_val]*4
df_y_vals[0].shape

(338, 1)

In [9]:
df_X_train = readCsvs('train')
df_X_train[0]['data'].shape

(550, 148)

In [10]:
df_y_train = pd.read_csv(PATH_DATASETS + 'Y_train.csv', index_col = 0)
df_y_trs = [df_y_train]*4
df_y_trs[0].shape

(550, 1)

#### I have to implement a fucntion in the 1.ramdom forest parameter tunning,  in order to obtain a csv with number of rois, number of n_estimator. and the accuracy

#### accuracy over validation test and not over oob score

In [11]:
rois_nEstimator_accuracy = [
    {'name' : 'area', 'data': [20, 430, 0.7367]},
    {'name' : 'meanCurv', 'data': [45, 965, 0.6893]},
    {'name' : 'thickness', 'data': [60, 945, 0.7012]},
    {'name' : 'volume', 'data': [20, 245,  0.7515]},
]

#rois_nEstimator_accuracy = function for read the csv with the information

In [14]:
accuracy = [] # {name: feature, data: {feature_list: feature list, accuracy_from_feature}}
P_value = [] # {name: feature, Data:p-value }

with futures.ProcessPoolExecutor() as executor:
    for (feature_train,
         feature_validate, 
         feature_info, df_y_tr, 
         df_y_val, result) in zip(df_X_train,
                                  df_X_val,
                                  rois_nEstimator_accuracy,
                                  df_y_trs, df_y_vals, 
                                  executor.map(PermutationTest,
                                               df_X_train,df_X_val,
                                               rois_nEstimator_accuracy,
                                               df_y_trs, 
                                               df_y_vals)):
        accuracy.append(result[0])
        P_value.append(result[1])
    

******************************** PERMUTATION TEST FOR: area ******************************
******************************** PERMUTATION TEST FOR: meanCurv ******************************
******************************** PERMUTATION TEST FOR: thickness ******************************
******************************** PERMUTATION TEST FOR: volume ******************************
volume
0 0.7041420118343196
area
0 0.7189349112426036
volume
1 0.7100591715976331
volume
2 0.7337278106508875
area
1 0.7218934911242604
thickness
0 0.6272189349112426
meanCurv
0 0.6331360946745562
volume
3 0.7130177514792899
volume
4 0.7130177514792899
area
2 0.6923076923076923
volume
5 0.6804733727810651
area
3 0.7011834319526628
thickness
1 0.6568047337278107
volume
6 0.6982248520710059
meanCurv
1 0.5769230769230769
volume
7 0.6952662721893491
area
4 0.7159763313609467
volume
8 0.6686390532544378
volume
9 0.7189349112426036
area
5 0.6834319526627219
thickness
2 0.6627218934911243
meanCurv
2 0.6183431952662722
volume

area
74 0.7337278106508875
volume
125 0.6775147928994083
volume
126 0.7307692307692307
area
75 0.7189349112426036
meanCurv
35 0.6479289940828402
volume
127 0.7011834319526628
thickness
36 0.6479289940828402
volume
128 0.7130177514792899
area
76 0.7041420118343196
volume
129 0.7189349112426036
volume
130 0.6893491124260355
area
77 0.7248520710059172
meanCurv
36 0.5887573964497042
volume
131 0.7011834319526628
thickness
37 0.6479289940828402
area
78 0.7189349112426036
volume
132 0.7100591715976331
volume
133 0.7100591715976331
area
79 0.6893491124260355
meanCurv
37 0.6183431952662722
volume
134 0.6863905325443787
thickness
38 0.6242603550295858
volume
135 0.7011834319526628
area
80 0.7396449704142012
volume
136 0.7337278106508875
area
81 0.7011834319526628
volume
137 0.7159763313609467
meanCurv
38 0.6272189349112426
volume
138 0.6952662721893491
thickness
39 0.591715976331361
area
82 0.727810650887574
volume
139 0.742603550295858
volume
140 0.6715976331360947
area
83 0.6834319526627219
v

volume
253 0.7041420118343196
thickness
72 0.6331360946745562
volume
254 0.6982248520710059
area
151 0.6863905325443787
meanCurv
71 0.6715976331360947
volume
255 0.727810650887574
volume
256 0.6952662721893491
area
152 0.7337278106508875
thickness
73 0.6449704142011834
volume
257 0.7248520710059172
area
153 0.7130177514792899
volume
258 0.7366863905325444
meanCurv
72 0.636094674556213
volume
259 0.7011834319526628
area
154 0.6923076923076923
volume
260 0.7130177514792899
thickness
74 0.6153846153846154
volume
261 0.7071005917159763
area
155 0.7248520710059172
meanCurv
73 0.5976331360946746
volume
262 0.6952662721893491
area
156 0.6982248520710059
volume
263 0.7307692307692307
thickness
75 0.5976331360946746
volume
264 0.7130177514792899
area
157 0.7130177514792899
volume
265 0.727810650887574
meanCurv
74 0.5976331360946746
volume
266 0.7248520710059172
area
158 0.7130177514792899
volume
267 0.7071005917159763
thickness
76 0.6153846153846154
area
159 0.6952662721893491
volume
268 0.6982

volume
380 0.7307692307692307
area
226 0.6863905325443787
volume
381 0.7041420118343196
meanCurv
107 0.6745562130177515
area
227 0.6834319526627219
volume
382 0.7041420118343196
thickness
109 0.6834319526627219
volume
383 0.7100591715976331
area
228 0.7307692307692307
volume
384 0.7130177514792899
volume
385 0.7071005917159763
meanCurv
108 0.6893491124260355
area
229 0.7100591715976331
thickness
110 0.6272189349112426
volume
386 0.7366863905325444
area
230 0.7248520710059172
volume
387 0.7159763313609467
volume
388 0.7071005917159763
area
231 0.7159763313609467
meanCurv
109 0.6183431952662722
volume
389 0.7189349112426036
thickness
111 0.6627218934911243
volume
390 0.7130177514792899
area
232 0.6952662721893491
volume
391 0.7071005917159763
area
233 0.7189349112426036
volume
392 0.6834319526627219
meanCurv
110 0.6124260355029586
thickness
112 0.6538461538461539
volume
393 0.6775147928994083
area
234 0.7159763313609467
volume
394 0.7159763313609467
volume
395 0.6952662721893491
area
235

volume
506 0.6834319526627219
area
301 0.7189349112426036
volume
507 0.7100591715976331
thickness
145 0.6301775147928994
volume
508 0.7100591715976331
area
302 0.7189349112426036
volume
509 0.727810650887574
meanCurv
143 0.6331360946745562
area
303 0.6804733727810651
volume
510 0.7189349112426036
thickness
146 0.650887573964497
volume
511 0.7337278106508875
area
304 0.7307692307692307
volume
512 0.7100591715976331
meanCurv
144 0.6094674556213018
area
305 0.7189349112426036
volume
513 0.7011834319526628
thickness
147 0.6568047337278107
volume
514 0.6923076923076923
area
306 0.6982248520710059
volume
515 0.7041420118343196
volume
516 0.6923076923076923
area
307 0.6982248520710059
meanCurv
145 0.6538461538461539
volume
517 0.7130177514792899
thickness
148 0.7041420118343196
area
308 0.7041420118343196
volume
518 0.6952662721893491
volume
519 0.6982248520710059
area
309 0.6834319526627219
volume
520 0.7071005917159763
meanCurv
146 0.6272189349112426
volume
521 0.6893491124260355
thickness


...

thickness
9466 0.650887573964497
meanCurv
9364 0.6124260355029586
thickness
9467 0.636094674556213
meanCurv
9365 0.6183431952662722
thickness
9468 0.6153846153846154
meanCurv
9366 0.5887573964497042
thickness
9469 0.5976331360946746
meanCurv
9367 0.6183431952662722
thickness
9470 0.6331360946745562
meanCurv
9368 0.5946745562130178
thickness
9471 0.5887573964497042
meanCurv
9369 0.6124260355029586
thickness
9472 0.6715976331360947
meanCurv
9370 0.6005917159763313
thickness
9473 0.6272189349112426
meanCurv
9371 0.6420118343195266
thickness
9474 0.6538461538461539
meanCurv
9372 0.6479289940828402
thickness
9475 0.650887573964497
meanCurv
9373 0.665680473372781
thickness
9476 0.6390532544378699
meanCurv
9374 0.6420118343195266
thickness
9477 0.6568047337278107
meanCurv
9375 0.606508875739645
thickness
9478 0.6715976331360947
meanCurv
9376 0.6242603550295858
thickness
9479 0.621301775147929
meanCurv
9377 0.636094674556213
thickness
9480 0.6242603550295858
meanCurv
9378 0.6094674556213018
th

meanCurv
9486 0.5621301775147929
thickness
9590 0.650887573964497
meanCurv
9487 0.6893491124260355
thickness
9591 0.6479289940828402
meanCurv
9488 0.6745562130177515
thickness
9592 0.6301775147928994
meanCurv
9489 0.5946745562130178
thickness
9593 0.636094674556213
meanCurv
9490 0.591715976331361
thickness
9594 0.5798816568047337
meanCurv
9491 0.6745562130177515
thickness
9595 0.7071005917159763
meanCurv
9492 0.606508875739645
thickness
9596 0.6301775147928994
meanCurv
9493 0.6449704142011834
thickness
9597 0.665680473372781
meanCurv
9494 0.6538461538461539
thickness
9598 0.6390532544378699
meanCurv
9495 0.636094674556213
thickness
9599 0.6242603550295858
meanCurv
9496 0.621301775147929
thickness
9600 0.621301775147929
meanCurv
9497 0.6390532544378699
thickness
9601 0.6627218934911243
meanCurv
9498 0.6183431952662722
thickness
9602 0.6479289940828402
meanCurv
9499 0.6538461538461539
thickness
9603 0.5887573964497042
meanCurv
9500 0.6183431952662722
thickness
9604 0.6893491124260355
mea

meanCurv
9609 0.6272189349112426
thickness
9713 0.6124260355029586
meanCurv
9610 0.6834319526627219
thickness
9714 0.6745562130177515
meanCurv
9611 0.6479289940828402
thickness
9715 0.6449704142011834
meanCurv
9612 0.6331360946745562
thickness
9716 0.7041420118343196
meanCurv
9613 0.6449704142011834
thickness
9717 0.6479289940828402
meanCurv
9614 0.636094674556213
thickness
9718 0.5976331360946746
meanCurv
9615 0.591715976331361
thickness
9719 0.606508875739645
meanCurv
9616 0.6390532544378699
thickness
9720 0.6686390532544378
meanCurv
9617 0.6331360946745562
thickness
9721 0.6479289940828402
meanCurv
9618 0.6035502958579881
thickness
9722 0.6301775147928994
meanCurv
9619 0.665680473372781
thickness
9723 0.6272189349112426
meanCurv
9620 0.665680473372781
thickness
9724 0.6420118343195266
meanCurv
9621 0.650887573964497
thickness
9725 0.665680473372781
meanCurv
9622 0.6597633136094675
thickness
9726 0.6390532544378699
meanCurv
9623 0.6124260355029586
thickness
9727 0.7041420118343196
me

thickness
9836 0.636094674556213
meanCurv
9732 0.665680473372781
thickness
9837 0.6420118343195266
meanCurv
9733 0.6420118343195266
thickness
9838 0.6124260355029586
meanCurv
9734 0.621301775147929
thickness
9839 0.6863905325443787
meanCurv
9735 0.6420118343195266
thickness
9840 0.6390532544378699
meanCurv
9736 0.6183431952662722
thickness
9841 0.5857988165680473
meanCurv
9737 0.5946745562130178
thickness
9842 0.6331360946745562
meanCurv
9738 0.6035502958579881
thickness
9843 0.665680473372781
meanCurv
9739 0.6242603550295858
thickness
9844 0.6153846153846154
meanCurv
9740 0.6183431952662722
thickness
9845 0.6420118343195266
meanCurv
9741 0.6242603550295858
thickness
9846 0.5710059171597633
meanCurv
9742 0.6479289940828402
thickness
9847 0.6775147928994083
meanCurv
9743 0.6479289940828402
thickness
9848 0.5976331360946746
meanCurv
9744 0.6627218934911243
thickness
9849 0.6479289940828402
meanCurv
9745 0.6390532544378699
thickness
9850 0.6420118343195266
meanCurv
9746 0.6272189349112426

thickness
9959 0.6420118343195266
meanCurv
9855 0.636094674556213
thickness
9960 0.6479289940828402
meanCurv
9856 0.6331360946745562
thickness
9961 0.6627218934911243
meanCurv
9857 0.6153846153846154
thickness
9962 0.6449704142011834
meanCurv
9858 0.6390532544378699
thickness
9963 0.636094674556213
meanCurv
9859 0.621301775147929
thickness
9964 0.6597633136094675
meanCurv
9860 0.621301775147929
thickness
9965 0.5887573964497042
meanCurv
9861 0.6479289940828402
thickness
9966 0.5680473372781065
meanCurv
9862 0.6597633136094675
thickness
9967 0.5857988165680473
meanCurv
9863 0.7011834319526628
thickness
9968 0.6005917159763313
meanCurv
9864 0.5621301775147929
thickness
9969 0.6479289940828402
meanCurv
9865 0.6538461538461539
thickness
9970 0.6301775147928994
meanCurv
9866 0.6479289940828402
thickness
9971 0.6124260355029586
meanCurv
9867 0.650887573964497
thickness
9972 0.6893491124260355
meanCurv
9868 0.6301775147928994
thickness
9973 0.6775147928994083
meanCurv
9869 0.6479289940828402


In [13]:
def PermutationTest(feature_train, feature_validate, feature_info, df_y_tr, df_y_val):
    data_train = feature_train['data']
    data_validation = feature_validate['data']
    name = feature_info['name']
    
    permutations = 10000
    
    rois = feature_info['data'][0]
    n_estimators = feature_info['data'][1]
    accuracy_observed = feature_info['data'][2]
    
    accuracy_from_feature = []
    
    # random.sample: returns a particular length list of items chosen from the sequence
    feature_list = random.sample(list(data_train.columns.values), rois)
    #print(feature_list)
    
    #the amount of times when the permutation is major than the accuracu from the feature etracion with RF
    counter = 0 
    
    list_of_rois=[]
    
    print("******************************** PERMUTATION TEST FOR: " + name + " ******************************")
    for i in range (0,permutations):
        
        x_temp = data_train[feature_list].copy()
        #print(x_temp)
        
        # create RF classifier
        clf = RandomForestClassifier(n_estimators = n_estimators, random_state= 0 , n_jobs= -1,verbose= False, oob_score= True)
        
        # Train Classifier
        clf.fit(x_temp, df_y_train['class'].values)
        
        # Prediction
        X_val_temp = data_validation[feature_list].copy()
        #print(name)
        #print(X_val_temp.shape)
        y_pred_val = clf.predict(X_val_temp)
        
        #score
        score = accuracy_score(y_pred_val, df_y_val['class'].values)
        if (score > accuracy_observed):
            counter += 1
        
        accuracy_from_feature.append({'feature_list':feature_list, 'score':score})
        list_of_rois.append(feature_list)
        
        #print(i, accuracy_from_feature[-1])
        print(name)
        print(i, score)
        
        # this is for change a new feature list.
        # this while verify that the new feature_list have not been used in a previous experiment
        while feature_list in list_of_rois:
            feature_list = random.sample(list(data_train.columns.values), rois)
            
    result = []
    result.append({'name' : name, 'data' : accuracy_from_feature})
    result.append({'name':name, 'data':(counter/permutations)})
    
    return result

    

In [15]:
print(P_value)

[{'name': 'area', 'data': 0.0532}, {'name': 'meanCurv', 'data': 0.0181}, {'name': 'thickness', 'data': 0.0105}, {'name': 'volume', 'data': 0.0086}]
